In [1]:
import tensorflow as tf
import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot
import sklearn.model_selection

from tensorflow.keras import backend as K

In [2]:
num_classes = 2
n = 10   # number of time steps

pattern = np.zeros((num_classes, n))

# sawtooth
x = []
y = 0.0
for i in range(n):
  x.append(y)
  if i < n/2:
    y += 0.2
  else:
    y -= 0.2
pattern[0] = x

# sin wave
pattern[1] = np.sin(np.linspace(0, np.pi*2, n))*0.5 + 0.5

instances_per_class = 150
m = num_classes * instances_per_class

X = np.zeros((m, n))
y = np.zeros(m)
for i in range(num_classes):
  for j in range(instances_per_class):
    X[i*instances_per_class+j] = pattern[i] + np.random.normal(scale=0.05, size=n)
    y[i*instances_per_class+j] = i

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.3, random_state=0)


In [3]:
def get_rnn(sequence_length=10):
  inputs = keras.layers.Input(shape=(sequence_length,1))
  x = keras.layers.SimpleRNN(1)(inputs)
  x = keras.layers.Dense(1, activation='sigmoid')(x)
  model = keras.models.Model(inputs, x)
  return model

model = get_rnn(10)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 10, 1)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ (None, 1)              │             3 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │             2 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5 (20.00 B)

 Trainable params: 5 (20.00 B)

 Non-trainable params: 0 (0.00 B)

In [4]:
K.clear_session()
model = get_rnn(10)
model.compile(loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=500,
  batch_size=16, validation_split=0.3)


Epoch 1/500
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.4762 - loss: 0.7121 - val_accuracy: 0.5079 - val_loss: 0.7001
Epoch 2/500
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4762 - loss: 0.7101 - val_accuracy: 0.5079 - val_loss: 0.6993
Epoch 3/500
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4762 - loss: 0.7092 - val_accuracy: 0.5079 - val_loss: 0.6989
Epoch 4/500
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4762 - loss: 0.7085 - val_accuracy: 0.5079 - val_loss: 0.6985
Epoch 5/500
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4762 - loss: 0.7080 - val_accuracy: 0.5079 - val_loss: 0.6979
Epoch 6/500
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4762 - loss: 0.7071 - val_accuracy: 0.5079 - val_loss: 0.6975
Epoch 7/500
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4762 - loss: 0.7064 - val_accuracy: 0.5079 - val_loss: 0.6968
Epoch 8/500
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4762 - loss: 0.7053 - val_accuracy: 0.5079 - 

In [5]:
rnn_weights   = model.get_layer('simple_rnn').get_weights()
dense_weights = model.get_layer('dense').get_weights()

print(rnn_weights)
print(dense_weights)

[array([[-4.476927]], dtype=float32), array([[-0.3025359]], dtype=float32), array([1.4782706], dtype=float32)]
[array([[-4.016388]], dtype=float32), array([-0.31879622], dtype=float32)]
